In [1]:
import os,sys
os.environ["CUDA_VISIBLE_DEVICES"]="1"
sys.path.append('../..')
# os.chdir("../tmp/HunyuanVideo")
sys.path.append('..')
sys.path.append('.')
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from einops import rearrange
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import animation
from IPython.display import HTML
import numpy as np

# save_path = os.path.join(output_dir, f"{file_name}.mp4")

def show_video(videos: torch.Tensor, save_path=None, rescale=True):
    for x in videos:
        x=x.cpu().float() # c t h w
        x=x.permute(1,2,3,0) # t h w c
        if rescale:
            x = (x + 1.0) / 2.0  # -1,1 -> 0,1
        x = torch.clamp(x, 0, 1)
        x = (x * 255).numpy().astype(np.uint8)
        fig = plt.figure()
        im = plt.imshow(x[0,:,:,:])
        def init():
            im.set_data(x[0,:,:,:])

        def animate(i):
            im.set_data(x[i,:,:,:])
            return im
        plt.close()
        anim = animation.FuncAnimation(fig, animate, init_func=init, frames=x.shape[0],
                                    interval=50)
        return HTML(anim.to_html5_video())

In [ ]:
import hyvideo.config
print(hyvideo.config.__file__)
from hyvideo.config import parse_args
string_args="""--video-size 720 1280 --video-length 129 --infer-steps 50 --prompt cat. --flow-reverse --use-cpu-offload --save-path ./results"""
string_args=string_args.split(" ")
print(string_args)
args = parse_args(string_args=string_args)
print(args)


In [ ]:
import os
import time
from pathlib import Path
from loguru import logger
from datetime import datetime
import torch

from hyvideo.utils.file_utils import save_videos_grid
from hyvideo.config import parse_args
from hyvideo.inference import HunyuanVideoSampler
from hyvideo.modules import load_model
from hyvideo.vae import load_vae
from hyvideo.vae.autoencoder_kl_causal_3d import AutoencoderKLCausal3D
from hyvideo.diffusion.pipelines import HunyuanVideoPipeline

# hunyuan_video_sampler = HunyuanVideoSampler.from_pretrained(models_root_path, args=args)
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_grad_enabled(False)
# model = load_model(
#     args,
#     in_channels=in_channels,
#     out_channels=out_channels,
#     factor_kwargs=factor_kwargs,
# )

t_ops_config_path="t_ops_config.json"

vae, _, s_ratio, t_ratio = load_vae(
            args.vae,
            args.vae_precision,
            logger=logger,
            device=device,
            t_ops_config_path=t_ops_config_path
        )
vae.to(device)
vae.eval()
vae.enable_tiling()
model=vae


In [52]:
vae.tile_overlap_factor=0


In [ ]:
from hyvideo.vae.adaptive_temporal_tiling import AdaptiveTemporalTiling

adaptor = AdaptiveTemporalTiling(
        vae_ckpt_path="ckpts/hunyuan-video-t2v-720p/vae",
        device=device,
        vae_precision="fp16",
    )

In [34]:
from torch.utils.data import DataLoader
from dataset_processor.dataset_loader import VideoTensorDataset

tensor_dir="video_data/processed_720p_tensors"
dataset = VideoTensorDataset(tensor_dir)
dataloader = DataLoader(dataset, batch_size=args.batch_size, shuffle=False, num_workers=4)

In [56]:
import torch.nn.functional as F
video_tensor, file_name=dataset[0]
video_tensor=F.interpolate(video_tensor, size=(720//2,1280//2))
video_tensor=video_tensor.unsqueeze(0)[:,:,:200]
video_tensor = video_tensor.to(device, dtype=torch.float16)

In [ ]:
print(video_tensor.shape,file_name)
# with torch.no_grad():
#     # Encode and decode video
#     reconstructed_video = model(
#         video_tensor,
#         return_dict=False,
#         return_posterior=True,
#         sample_posterior=False
#     )[0]

with torch.no_grad():
    # =========== Encode阶段 ===========
    posterior_out = model.temporal_tiled_encode(
        x=video_tensor,
        adaptor=adaptor,  # 传入我们自适应类
        return_dict=True
    )
    # posterior_out 是 AutoencoderKLOutput
    posterior = posterior_out.latent_dist  # 里面包含 mean/var



In [ ]:
with torch.no_grad():
# =========== Decode阶段 ===========
    reconstructed_video = model.temporal_tiled_decode(
        z=posterior.mode(),  # 或 sample() 取随机
        adaptor=adaptor,     # 同样传入
        return_dict=True
    ).sample
print(video_tensor.shape)
print(reconstructed_video.shape)
show_video(reconstructed_video)

In [59]:
# show_video(reconstructed_video)